In [1]:
# Lo primero sera importar las librerias y estableces algunas configuraciones para
# facilitar la lectura en pantalla

import pandas as pd
import numpy as np
import re
import unicodedata
import warnings
import operator
import nltk
nltk.download('cess_esp')
from nltk.corpus import cess_esp as cess
from nltk.tokenize import RegexpTokenizer
import difflib

warnings.filterwarnings('ignore')
pd.options.display.max_rows
pd.set_option('display.max_colwidth', -1)
pd.options.display.max_rows = 4000

[nltk_data] Downloading package cess_esp to /home/luis/nltk_data...
[nltk_data]   Unzipping corpora/cess_esp.zip.


In [2]:
# leer los datos y cargarlos en dataframe, validando la carga al presentar informacion basica
# del archivo
mensajes = pd.read_json('/home/luis/mensajes.data/mensajespd.json')

print(mensajes.shape)

# borramos registros duplicados desde los datos originales
mensajes.drop_duplicates(inplace=True)

#print(mensajes.head())
print("\n")
print("Cantidad de registros de mensajes = %3d" % mensajes.shape[0])
print("\n")
print(mensajes.columns)

(1197, 3)


Cantidad de registros de mensajes = 1101


Index(['Enviado/por', 'Fecha/Hora', 'Texto'], dtype='object')


In [4]:
# ahora se va a proceder a colocar en minuscula, todo el contenido de la variable "Texto"
# todo a minuscula

mensajes.Texto = mensajes.Texto.str.lower()

# y luego eliminamos todos los acentos 


def strip_accents(text):
    """
    Strip accents from input String.

    :param text: The input string.
    :type text: String.

    :returns: The processed String.
    :rtype: String.
    """
    try:
        text = unicode(text, 'utf-8')
    except (TypeError, NameError): # unicode is a default on python 3 
        pass
    text = unicodedata.normalize('NFD', text)
    text = text.encode('ascii', 'ignore')
    text = text.decode("utf-8")
    return str(text)

mensajes["Texto"] = mensajes["Texto"].apply(lambda tx: strip_accents(tx))



**el dataset requiere depuracion ya que multiples
 registros (algunos), contienen informacion de un solo mensaje, lo que distorciona la
 interpretacion de dichos mensajes
 primer paso ordenar el dataframe por "Enviado/por" y "Fecha/Hora"**

In [5]:
# ordenamos
mensajes.sort_values(by=["Enviado/por", "Fecha/Hora"])

# reacomodando indices
indice = list(range(mensajes.shape[0]))
mensajes.index = indice

# convertimos a formato de fecha la columna "Fecha/Hora"
mensajes["Fecha/Hora"] = pd.to_datetime(mensajes["Fecha/Hora"])

# variable de interes tiempo en segundos (hora del dia en segundos)

mensajes['ts'] = mensajes["Fecha/Hora"].apply(lambda x: x.hour*60*60+x.minute*60+x.second)

# el tiempo entre mensajes: tem
# voala! funciona.. 

mensajes["tem"] = np.zeros(mensajes.shape[0], dtype=int)

mensajes["tem"] = abs(mensajes["ts"].diff())



In [6]:
%%time
# con este arreglo de indice calculado a continuacion, concatenamos 
# mensajes divididos en la fila i-1 y posterior eliminamos los (i) ndice
# ya q seran redundantes
# hay q tener en cuenta filtrar q mensajes a concatenar sean del mismo
# sender y que contenido de mensajes sean distintos

# cosideramos solo aquellos mensajes sucesivos que hayan sido recibidos
# en un intervalo de tiempo menor a 10 segundos ya que en este subgrupo se encuentran
# los mensajes divididos


indimulmensajes = mensajes[mensajes["tem"]<10].index

# uniendo y borrando registros redundantes, despues de unir mensajes fragmentados.
# Hay q recorrer el dataset de abajo hacia arriba (reverse) dado que: puede haber 
# mensajes fraccionados en mas de 2 partes y a la vez es posible borrar cada
# registro redundante una vez que se unan.


for i in reversed(indimulmensajes):
    if operator.and_(mensajes["Enviado/por"][i-1]==mensajes["Enviado/por"][i], mensajes["Texto"][i-1]!=mensajes["Texto"][i]):
        mensajes["Texto"][i-1] += mensajes["Texto"][i]
        mensajes.drop(mensajes.index[i], inplace = True)


# reacomodando indices
indice = list(range(mensajes.shape[0]))
mensajes.index = indice
        
       
# guardamos todo el nuevo dataframe con datos limpios, preprocesados
mensajes.to_pickle("/home/luis/mensajes.data/mensajes.pkl")

CPU times: user 9.27 s, sys: 0 ns, total: 9.27 s
Wall time: 9.45 s


In [9]:
# cargamos el dataframe mensajes con datos limpios, preprocesados

mensajes = pd.read_pickle("/home/luis/mensajes.data/mensajes.pkl")

# y creamos dos nuevas variables, una que indique si "Texto" proviene de un suscriptor del servicio
# de internet (0) o no (1) y otra que partiendo del contenido de "Texto" clasifique en mensaje,
# lo cual se hara luego de analizar la variable "Texto" mas adelante

# por defecto ninguno es suscriptor
mensajes["es_suscriptor"] = np.full(mensajes.shape[0], 9)

# aun no se a establecido criterio para clasificar, se inicializa en 0
mensajes["tipo_mensaje"] = np.full(mensajes.shape[0], 9)
# donde: 0->usuario reporta falla, 1->usuario solicita informacio cuota del servicio,
# 2->todo lo demas (no suscriptores)

# leer telefonos conocidos de clientes desde archivo
# al archivo tlfclientes.csv se le agrego un caracter alfanumerico al comienzo
# de cada registro para que pandas leyera como texto y no como numero

telef_clientes = pd.read_csv('/home/luis/mensajes.data/tlfclientes.csv')

# ahora se convierte telef_clientes.telef eliminando los caracteres- al inicio y final
telef_clientes["telef"] = telef_clientes["telef"].str.replace("-","")

# y se carga a memoria como lista, ya q son pocos
lista_telef = telef_clientes["telef"].tolist()

# ahora clasificamos como 0 la variable "es_suscriptor", si coincide lista con 
# "Enviado/por" del dataframe

mensajes.loc[mensajes["Enviado/por"].isin(lista_telef), "es_suscriptor"] = 0 

mensajes.loc[mensajes['Enviado/por'].str.len()<11, "es_suscriptor" ] = 1

mensajes.loc[mensajes['Enviado/por'].str.len()<11, "tipo_mensaje" ] = 2

In [11]:
%%time


tokenizer = RegexpTokenizer(r'\w+')


palab_clase1 = ["cuanto", "mes", "mensualidad", "interne", "wifi", "pago"]


# esta funcion compara las palabras clave (variantes) que deberia contener el
# mensaje de texto para considerarlo como categoria 1 (cliente solicita info
# acerca del cuota de servicio)
# funciona, hay que variar el cutoff para aumentar o disminuir la presicion
# en spyder se obtienen buenos resultados ajustando el valor a 0.7 pero
# aca se debe ajustar a 0.8 para reducir falsos positivos

def comparacion(a, b):
    if a==b: return True 
    else: return not(b in cess.words())

def sol_cuota(text, variantes):
    tokens = tokenizer.tokenize(text)
    
    palabras_en = []
    for palabra in variantes:
        var = difflib.get_close_matches(palabra, tokens, n=1, cutoff=0.8)
        # aqui se puede mejorar la presicion aplicando otro filtro
        # comparando var[0] mejor coicidencia con diccionario de 
        # palabras si se encuentra y es diferente se descarta {no sirvio :( }
        if len(var)>0 and len(str(var[0]))>2: #and comparacion(palabra, var[0]):
            palabras_en.append(var[0])
    return (len(palabras_en)>=2)

var1 = mensajes.index[mensajes["Texto"].apply(lambda x: sol_cuota(x, palab_clase1))]

# manualmente se obtuvo los mensajes que encajan con el criterio y no se
# clasifican como tipo 1 (en tipo_mensaje)

var2 = [91, 114, 150, 151, 214, 380, 419, 443, 467, 579, 678, 576, 788, 873]

# a var1 le quitamos var2 y el resultado se clasifica como tipo 1 (en tipo_mensaje)
# hay q anotar que en mensajes tipo 1 se encuentran tambien los reportes de pago
# los cual implica que la respuesta automatica debe indicar "ademas" que se: 
# haga reporte por otra via o que espere para validar.

var1 = set(var1)
var2 = set(var2)
var1 = var1-var2
var1 = list(var1)

mensajes.loc[mensajes.index[var1], "tipo_mensaje" ] = 1
mensajes.loc[mensajes.index[var1], "es_suscriptor" ] = 0

CPU times: user 368 ms, sys: 924 µs, total: 369 ms
Wall time: 416 ms


In [15]:
# aprovechamos var2 y extraemos los mensajes que son de no suscriptores en este 
# dominio

var2 = list(var2)
var2 = set(var2)
var3 = {114, 576, 579, 788}
var2 = var2-var3
var2 = list(var2)

mensajes.loc[mensajes.index[var2], "es_suscriptor" ] = 0

In [19]:
# ahora acemos igual con tipo mensaje 0 'reporte de falla de servicio'

mensajes.iloc[101:200]

,Enviado/por,Fecha/Hora,Texto,ts,tem,es_suscriptor,tipo_mensaje
101,04169298551,2018-07-09 15:11:41,ok ya esta aqui gracias,54701,2017.0,9,9
102,04263261041,2018-07-09 12:29:45,hola en q precio esta el internet para cancelar,44985,9716.0,9,9
103,04263261041,2018-07-09 12:38:25,para pagar antes de q el sr emilio cierre,45505,520.0,9,9
104,04265727738,2018-07-09 15:34:13,"buenas tardes sr luis, disculpe nada q tengo internet. soy la sra ildred",56053,10548.0,0,9
105,04265727738,2018-07-09 16:34:30,sr luis ya vamos para la casa,59670,3617.0,0,9
106,04265727738,2018-07-09 18:03:24,disculpe ya logro conectar el internet? aqui aun no podemos conectarnos . gracias,65004,5334.0,0,9
107,04265727738,2018-07-09 18:45:06,gracias,67506,2502.0,0,9
108,04267293055,2018-07-09 21:33:23,jetza venga que tia lla se vino,77603,10097.0,9,9
109,2482,2018-07-09 12:32:36,estrenalo tu y disfruta de la promocion 20- 20-20 en el salon rio manzanares del gran melia caracas. reserva ya al 0212-762-81-11 ext. 4051,45156,32447.0,1,2
110,44647,2018-07-09 16:22:35,g-762989 es tu c?digo de verificaci?n de google.,58955,13799.0,1,2


**es_suscriptor: esta variable toma valor 0 si el contenido de "Texto" proviene de un
cliente de la red comunitaria, y 1 en caso contrario. Pero ¿Como se determina si el 
mensaje proviene o no de un suscriptor?**
**a vuelo de pajaro podemos decir que:**

**1- Si hacemos query de "Enviado/por" en la base de datos de cliente y aparece en esta
definitivamente el mensaje fue enviado por un suscriptor, es el caso mas sencillo**

**2- Que pasa si el query no devuelve datos, quiere decir que no se encuentra registrado
pero esto no quiere decir que el mensaje no provenga de un cliente, sino que tal vez
este usando otro dispositivo no registrado en la base de datos, de alli que surga la necesidad
de analizar el contenido para establecer el valor de la variable es_suscriptor.
en este apartado se puede inferir que tal vez el usuario se identifique con su nombre
dentro del contenido del mensaje o tal vez no, en caso de que si, la solucion pasaria algo
similar a 1, en caso contrario se necesitara establecer mas precision en cuanto al contenido
de "Texto" y hasta sera necesario interpretar varios mensajes simultaneamente del mismo 
Enviado/por" en un tiempo-estipulado e interactuar con el remintente para clasificarlo.**

**3- Otro aspecto que pareciera ser prometedor es la frecuencia de origen de sms, obtenida de 
la variable "Enviado/por"**



**Bien, repasando de nuevo, un mensaje se considera que proviene de un suscriptor en estos 
casos:**

**1- su origen plasmado en la variable "Enviado/por", resuelto anteriormente**

**2- contenga frases asociadas con el servicio de internet, en especial las siguientes
son trascendentes: "no hay conexion", "no hay internet", "cuanto es la mensualidad"**

**3- tomar en cuenta comodin por error ortografico comun "interne" y otros
